In [1]:
!pip install nltk

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize
import transformers
import torch
from transformers import pipeline

nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
def chunk_transcript(data: pd.DataFrame) -> pd.DataFrame:
    
    # Ensure 'Transkript' column has no NaN values
    data['Transkript'] = data['Transkript'].fillna("").astype(str)

    # Chunking logic
    chunked_data = []
    current_speaker = None
    current_text = ""
    initial_timestamp = ""

    for index, row in data.iterrows():
        speaker = row['Sprecher']
        transcript = row['Transkript']
        timestamp = row['Timecode'] 

        if speaker != current_speaker:
            # Save previous chunk if exists
            if current_speaker is not None:
                chunked_data.append({
                    'Speaker': current_speaker,
                    'Transcript': current_text.strip(),
                    'Initial_Timestamp' : initial_timestamp,
                    'Current_Timestamp' : timestamp
                })
            
            # Start a new chunk
            current_speaker = speaker
            current_text = transcript
            initial_timestamp = timestamp 
        else:
            # Continue appending to the same speaker's chunk
            current_text += " " + transcript if isinstance(transcript, str) else ""

    # Save the last chunk
    if current_speaker is not None:
        chunked_data.append({
            'Speaker': current_speaker,
            'Transcript': current_text.strip(),
            'Initial_Timestamp' : initial_timestamp,
            'Current_Timestamp' : timestamp
        })

    # Convert to DataFrame
    chunked_df = pd.DataFrame(chunked_data)
    
    return chunked_df

In [4]:
def chunk_by_sentence(chunked_df: pd.DataFrame, min_tokens=256, max_tokens=512) -> pd.DataFrame:
    """
    Further splits chunks by sentence while ensuring each chunk is within a token range.

    Args:
        chunked_df (pd.DataFrame): Input DataFrame with 'Speaker' and 'Transcript' columns.
        min_tokens (int): Minimum number of tokens per chunk.
        max_tokens (int): Maximum number of tokens per chunk.

    Returns:
        pd.DataFrame: DataFrame with sentence-based chunked transcripts.
    """

    # Function to count tokens (approximate, assuming 1 word ≈ 1.2 tokens)
    def count_tokens(text):
        return len(text.split()) * 1.2  # Rough estimate

    # Initialize list for final merged chunks
    merged_chunks = []
    temp_chunk = []
    temp_token_count = 0
    speaker = None
    initial_timestamp = ""
    final_timestamp = ""

    # Process each row
    for _, row in chunked_df.iterrows():
        sentence = row['Transcript']
        sentence_tokens = count_tokens(sentence)

        # If adding this chunk keeps us within MAX_TOKENS
        if temp_token_count + sentence_tokens <= max_tokens:
            if not temp_chunk:
                speaker = row['Speaker']  # Store speaker only for new chunks
                initial_timestamp = row['Initial_Timestamp']
            temp_chunk.append(sentence)
            temp_token_count += sentence_tokens
        else:
            # Save the previous chunk before starting a new one
            if temp_chunk:
                merged_chunks.append({
                    'Speaker': speaker,
                    'Transcript': " ".join(temp_chunk),
                    'Initial_Timestamp' : initial_timestamp,
                    'Current_Timestamp' : row['Current_Timestamp']
                })

            # Start a new chunk with the current sentence
            temp_chunk = [sentence]
            temp_token_count = sentence_tokens
            speaker = row['Speaker']
            initial_timestamp = row['Initial_Timestamp']
            final_timestamp = row['Current_Timestamp']

    # Save last chunk if any content remains
    if temp_chunk:
        merged_chunks.append({
            'Speaker': speaker,
            'Transcript': " ".join(temp_chunk),
            'Initial_Timestamp' : initial_timestamp,
            'Current_Timestamp' : final_timestamp
        })

    # Convert to DataFrame
    final_merged_df = pd.DataFrame(merged_chunks)
    
    return final_merged_df

In [5]:
model_path = "models/meta-llama/Llama-3.3-70B-Instruct"
quantization_config = transformers.BitsAndBytesConfig(llm_int8_enable_fp32_cpu_offload=True)
model = transformers.AutoModelForCausalLM.from_pretrained(model_path, device_map="auto", torch_dtype=torch.bfloat16, quantization_config=quantization_config)
tokenizer = transformers.AutoTokenizer.from_pretrained(model_path)
nlp_pipeline = pipeline("text-generation", model, tokenizer=tokenizer)

Loading checkpoint shards:   0%|          | 0/30 [00:00<?, ?it/s]

In [6]:
nlp_pipeline

In [ ]:
def extract_metadata(model_name: str, chunks: pd.DataFrame) -> pd.DataFrame:
    """
    Extract metadata from multiple chunks of a German transcript using the specified model.
    
    Args:
        client: The Groq client object.
        model_name: The name of the model to use (e.g., 'llama3-8b-8192').
        chunks: A DataFrame with 'Speaker' and 'Transcript' columns.
    
    Returns:
        A DataFrame containing the extracted metadata for all chunks.
    """
    all_metadata = []
    num_chunks = len(chunks)

    for i, row in chunks.iloc[:5].iterrows():  # Iterate over DataFrame rows
        speaker = row["Speaker"]
        transcript = row["Transcript"]
        timestamp = row['Timestamp']

        print(f"Processing chunk {i+1}/{len(chunks)} for Speaker: {speaker}...")

        # Prompt for metadata extraction
        prompt = f"""
Extrahieren Sie die folgenden Informationen aus dem Transkript und geben Sie die Antwort auf Deutsch ein. 
Wenn die Antwort nicht gefunden wurde, geben Sie stattdessen %%% zurück. 
Fügen Sie nach jedem Wert in Klammern den entsprechenden {timestamp} hinzu. 
aber nur, wenn der Wert nicht %%% ist und den Wert aus dem Kontext behalten, aber den Zeitstempel nicht hinzufügen. Fügen Sie keine Präambel ein.


Transkript:
{transcript}

Metadaten columns to be present:
        NAME: 
        JAHRGANG: 
        ORT: 
        GESCHLECHT: 
        BERUF: 
        VAT_JG: 
        VAT_KONFESSION: 
        VAT_HERKUN: 
        VAT_SCHULE: 
        VAT_AUSBIL: 
        VAT_STAND: 
        VAT_POLOR:

Fügen Sie keine Präambel ein.
"""


        # Generate response using the Groq client
        response = nlp_pipeline(prompt, max_new_tokens=200, truncation=True)[0]["generated_text"]

        # Extract metadata from the response
        metadata = response
        # Parse metadata into a dictionary
        extracted_metadata = {"Speaker": speaker}  # Store speaker info
        for line in metadata.split("\n"):
            if ":" in line:
                key, value = line.split(":", 1)
                extracted_metadata[key.strip()] = value.strip()

        # Remove "%%%%" values before storing
        extracted_metadata = {k: (v if v != "%%%" else "") for k, v in extracted_metadata.items()}

        if i < 5 - 1:  # If not the last iteration
            extracted_metadata = {k: v.strip() + "," if v else v.strip() for k, v in extracted_metadata.items()}

        # Append metadata for this chunk
        all_metadata.append(extracted_metadata)

    # Convert metadata list into a DataFrame
    return pd.DataFrame(all_metadata)


In [35]:
import os
import time

folder_path = "Transcripts"
MODEL = "llama-3.3-70b-versatile"

# List to store metadata for all files
all_metadata = []
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path) & filename.endswith(".csv"):
        print(f"\nProcessing file: {filename}")
        input_data = pd.read_csv(file_path, sep=None, engine='python')
        speaker_chunks_df = chunk_transcript(input_data)  # Stores speaker-based chunks
        final_chunks_df = chunk_by_sentence(speaker_chunks_df)
        final_chunks_df['Timestamp'] = final_chunks_df['Initial_Timestamp'] + " - " + final_chunks_df['Current_Timestamp'] 
        final_chunks_df.drop(columns=['Initial_Timestamp', "Current_Timestamp"], inplace=True)
        # print(final_chunks_df)
        

        # Extract metadata for the chunks
        llama_70b_responses = extract_metadata(nlp_pipeline, final_chunks_df)
        # Ensure that the response DataFrame contains metadata columns
        if not llama_70b_responses.empty:
            # Merge chunk outputs into a single row 
            merged_metadata = llama_70b_responses.apply(lambda col: ' '.join(col.dropna().astype(str)))
            
            for column in merged_metadata.index:
                unique_values = set([value.strip() for value in merged_metadata[column].strip().split(",")])
                list_unique_values = list(filter(None, unique_values))
                merged_metadata[column] = " | ".join(list_unique_values)

            # Add filename for reference
            # merged_metadata["Filename"] = filename  

            # Append to list
            all_metadata.append(merged_metadata)
        else:
            print(f"No metadata extracted from {filename}")
         
        time.sleep(0.5)

# Convert list of metadata rows into a single DataFrame
final_metadata_df = pd.DataFrame(all_metadata)



Processing file: adg0001_er_2024_10_31.csv
Processing chunk 1/21 for Speaker: INT_AH...
Processing chunk 2/21 for Speaker: IP_FA...
Processing chunk 3/21 for Speaker: INT_AH...
Processing chunk 4/21 for Speaker: IP_FA...
Processing chunk 5/21 for Speaker: INT_AH...


In [40]:
final_metadata_df

,Speaker,Fügen Sie nach jedem Wert in Klammern den entsprechenden 00,Transkript,Metadaten columns to be present,NAME,JAHRGANG,ORT,GESCHLECHT,BERUF,VAT_JG,VAT_KONFESSION,VAT_HERKUN,VAT_SCHULE,VAT_AUSBIL,VAT_STAND,VAT_POLOR,"Der Tagesablauf, ja halt wecken, wie früher im Landjahr auch, um 6 Uhr wurde geweckt, im Winter um 7 Uhr, und dann war normalerweise Frühsport. Entweder sind wir ..., im Sommer haben wir draußen Frühsport gemacht und im Winter im ..., also wir haben schon mal einen Lauf gemacht oder so Freiübungen und im Winter, dann haben wir das halt im Hof nur so Freiübungen gemacht und keine Läufe. Das hat so vielleicht sagen wir mal 10 Minuten bis eine Viertelstunde. Und dann ging’s zum Waschen, in die Waschräume, in die Duschräume und anschließend war Frühstück. Ja nach dem Frühstück wurde dann die Arbeit verteilt. Die ersten 4 Wochen blieb man halt im Lager. Da wurde man vertraut gemacht mit eben all den Dingen, mit den Tageseinteilungen. Und dann wurde eingeteilt zum Hausdienst, also, sagen wir mal, sauber machen oder Waschküchengruppe, Bügelgruppe oder Küche und auch Verwaltungsarbeiten. Also im Büro mussten wir auch helfen. Eben, sagen wir mal, es wurde jeder getestet nach seinen Fähigkeiten und auch mehr oder weniger denn da eingesetzt. Er musste zwar alles lernen, er kam überall mit rein aber, ich meine, viele schimpfen auf den Arbeitsdienst und sagen, das war Mist, das war Blödsinn. Vielleicht lag’s an der Führung aber, wie gesagt, wir hatten das Glück, wir hatten ne wirklich, also sehr gute Führerin, die äußerst korrekt war. Also wirklich äußerst korrekt. Die weder Entgleisungen duldete, noch sich selbst, sagen wir mal, irgendwie gehen ließ. Die auch sah, wenn irgendwelche Ungerechtigkeiten waren. Zum Beispiel"
0,INT_AH | IP_FA,07:22.00 - 00:16:30.00 hinzu. | 00:06.00 - 00:...,,,%%% (00:07:22.00 - 00:16:30.00) | %%% (00:04:1...,%%% (00:07:22.00 - 00:16:30.00) | %%% (00:04:1...,%%% (00:04:11.00 - 00:07:23.00) | Mülheim an d...,%%% (00:07:22.00 - 00:16:30.00) | %%% (00:04:1...,Büro (00:04:11.00 - 00:07:23.00) | %%% (00:07:...,42 (00:07:22.00 - 00:16:30.00) | %%% (00:04:11...,%%% (00:07:22.00 - 00:16:30.00) | %%% (00:04:1...,%%% (00:07:22.00 | %%% (00:04:11.00 - 00:07:23...,,,,,ans Bett gebracht | entgleiste die Lokomotive ...


In [13]:
final_metadata_df.to_csv("metadata_results.csv", index=False)